In [23]:
import pandas as pd
df = pd.read_csv("csv_output/actor.csv")

In [26]:
df.isnull().count()

actor_id              200
first_name            200
last_name             200
last_update_year      200
last_update_month     200
last_update_day       200
last_update_hour      200
last_update_minute    200
last_update_second    200
dtype: int64

In [1]:
import sqlite3

# Connect to the database
database_path = "sqlite-sakila.sqlite"
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Step 1: Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Step 2: For each table, get column names and types
for table_name, in tables:
    print(f"\nTable: {table_name}")
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    # PRAGMA table_info gives: (cid, name, type, notnull, dflt_value, pk)
    for col in columns:
        col_name = col[1]
        col_type = col[2]
        print(f"  {col_name} : {col_type}")

conn.close()


Table: actor
  actor_id : numeric
  first_name : VARCHAR(45)
  last_name : VARCHAR(45)
  last_update : TIMESTAMP

Table: country
  country_id : SMALLINT
  country : VARCHAR(50)
  last_update : TIMESTAMP

Table: city
  city_id : INT
  city : VARCHAR(50)
  country_id : SMALLINT
  last_update : TIMESTAMP

Table: address
  address_id : INT
  address : VARCHAR(50)
  address2 : VARCHAR(50)
  district : VARCHAR(20)
  city_id : INT
  postal_code : VARCHAR(10)
  phone : VARCHAR(20)
  last_update : TIMESTAMP

Table: language
  language_id : SMALLINT
  name : CHAR(20)
  last_update : TIMESTAMP

Table: category
  category_id : SMALLINT
  name : VARCHAR(25)
  last_update : TIMESTAMP

Table: customer
  customer_id : INT
  store_id : INT
  first_name : VARCHAR(45)
  last_name : VARCHAR(45)
  email : VARCHAR(50)
  address_id : INT
  active : CHAR(1)
  create_date : TIMESTAMP
  last_update : TIMESTAMP

Table: film
  film_id : INT
  title : VARCHAR(255)
  description : BLOB SUB_TYPE TEXT
  release_year

In [ ]:


# Example usage
replacements = export_sqlite_to_csv("sqlite-sakila.sqlite", "sqlite-sakila")
print("\nReplaced timestamp columns:\n", replacements)

Processing table: actor
Saved actor -> sqlite-sakila\actor.csv
Processing table: country
Saved country -> sqlite-sakila\country.csv
Processing table: city
Saved city -> sqlite-sakila\city.csv
Processing table: address
Saved address -> sqlite-sakila\address.csv
Processing table: language
Saved language -> sqlite-sakila\language.csv
Processing table: category
Saved category -> sqlite-sakila\category.csv
Processing table: customer
Saved customer -> sqlite-sakila\customer.csv
Processing table: film
Saved film -> sqlite-sakila\film.csv
Processing table: film_actor
Saved film_actor -> sqlite-sakila\film_actor.csv
Processing table: film_category
Saved film_category -> sqlite-sakila\film_category.csv
Processing table: film_text
Saved film_text -> sqlite-sakila\film_text.csv
Processing table: inventory
Saved inventory -> sqlite-sakila\inventory.csv
Processing table: staff
Saved staff -> sqlite-sakila\staff.csv
Processing table: store
Saved store -> sqlite-sakila\store.csv
Processing table: paym

In [37]:
name = "sqlite-sakila"
database_path = name + ".sqlite"
conn = sqlite3.connect(database_path)

# Step 1: List tables
tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables = tables_df['name'].tolist()

schema_info = {}

for table in tables:
    # Get columns and types
    columns = pd.read_sql(f"PRAGMA table_info({table});", conn)
    # Get foreign keys
    foreign_keys = pd.read_sql(f"PRAGMA foreign_key_list({table});", conn)

    schema_info[table] = {
        "columns": columns[['name', 'type']].to_dict(orient="records"),
        "primary_keys": columns[columns['pk'] > 0]['name'].tolist(),
        "foreign_keys": foreign_keys[['from', 'table', 'to']].to_dict(orient="records")
    }

conn.close()
print(schema_info)

{'actor': {'columns': [{'name': 'actor_id', 'type': 'numeric'}, {'name': 'first_name', 'type': 'VARCHAR(45)'}, {'name': 'last_name', 'type': 'VARCHAR(45)'}, {'name': 'last_update', 'type': 'TIMESTAMP'}], 'primary_keys': ['actor_id'], 'foreign_keys': []}, 'country': {'columns': [{'name': 'country_id', 'type': 'SMALLINT'}, {'name': 'country', 'type': 'VARCHAR(50)'}, {'name': 'last_update', 'type': 'TIMESTAMP'}], 'primary_keys': ['country_id'], 'foreign_keys': []}, 'city': {'columns': [{'name': 'city_id', 'type': 'INT'}, {'name': 'city', 'type': 'VARCHAR(50)'}, {'name': 'country_id', 'type': 'SMALLINT'}, {'name': 'last_update', 'type': 'TIMESTAMP'}], 'primary_keys': ['city_id'], 'foreign_keys': [{'from': 'country_id', 'table': 'country', 'to': 'country_id'}]}, 'address': {'columns': [{'name': 'address_id', 'type': 'INT'}, {'name': 'address', 'type': 'VARCHAR(50)'}, {'name': 'address2', 'type': 'VARCHAR(50)'}, {'name': 'district', 'type': 'VARCHAR(20)'}, {'name': 'city_id', 'type': 'INT'},

In [4]:
import sqlite3
import pandas as pd
import os
import json

def erd_text_generate(name, replacement=None):
    """
    Generate schema information and ERD text from a SQLite database.

    Args:
        name (str): database name (without .sqlite extension).
        replacement (dict): {table: {original_col: [expanded_cols...]}}
                            If provided, schema_info will replace those columns,
                            and ERD text will be generated from updated schema_info.
    """
    database_path = name + ".sqlite"
    conn = sqlite3.connect(database_path)

    # Step 1: List tables
    tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
    tables = tables_df['name'].tolist()

    schema_info = {}

    for table in tables:
        # Get columns and types
        columns = pd.read_sql(f"PRAGMA table_info({table});", conn)
        # Get foreign keys
        foreign_keys = pd.read_sql(f"PRAGMA foreign_key_list({table});", conn)

        # Prepare column list
        col_list = []
        for _, row in columns.iterrows():
            col_name, col_type = row["name"], row["type"]

            if replacement and table in replacement and col_name in replacement[table]:
                # Replace with expanded columns
                for exp_col in replacement[table][col_name]:
                    col_list.append({"name": exp_col, "type": "INT"})
            else:
                col_list.append({"name": col_name, "type": col_type})

        schema_info[table] = {
            "columns": col_list,
            "primary_keys": columns[columns['pk'] > 0]['name'].tolist(),
            "foreign_keys": foreign_keys[['from', 'table', 'to']].to_dict(orient="records")
        }

    conn.close()

    # ---- Generate ERD text from updated schema_info ----
    def generate_erd_text(schema_info):
        erd_text = ""
        for table, info in schema_info.items():
            erd_text += f"\nTable: {table}\n"
            erd_text += "Columns:\n"
            for col in info["columns"]:
                erd_text += f"  - {col['name']} ({col['type']})\n"

            if info["primary_keys"]:
                erd_text += f"Primary Key: {', '.join(info['primary_keys'])}\n"
            if info["foreign_keys"]:
                erd_text += "Foreign Keys:\n"
                for fk in info["foreign_keys"]:
                    erd_text += f"  - {fk['from']} → {fk['table']}.{fk['to']}\n"
        return erd_text

    erd_text = generate_erd_text(schema_info)

    return schema_info, erd_text

schema_info, erd_text = erd_text_generate("sqlite-sakila", replacements)

In [6]:
print(erd_text)


Table: actor
Columns:
  - actor_id (numeric)
  - first_name (VARCHAR(45))
  - last_name (VARCHAR(45))
  - last_update_year (INT)
  - last_update_month (INT)
  - last_update_day (INT)
  - last_update_hour (INT)
  - last_update_minute (INT)
  - last_update_second (INT)
Primary Key: actor_id

Table: country
Columns:
  - country_id (SMALLINT)
  - country (VARCHAR(50))
  - last_update_year (INT)
  - last_update_month (INT)
  - last_update_day (INT)
  - last_update_hour (INT)
  - last_update_minute (INT)
  - last_update_second (INT)
Primary Key: country_id

Table: city
Columns:
  - city_id (INT)
  - city (VARCHAR(50))
  - country_id (SMALLINT)
  - last_update_year (INT)
  - last_update_month (INT)
  - last_update_day (INT)
  - last_update_hour (INT)
  - last_update_minute (INT)
  - last_update_second (INT)
Primary Key: city_id
Foreign Keys:
  - country_id → country.country_id

Table: address
Columns:
  - address_id (INT)
  - address (VARCHAR(50))
  - address2 (VARCHAR(50))
  - district (VAR